<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>
<h2 style="color:rgb(0,120,170)">Transfer Learning</h2>

In [1]:
#!pip install -q -U tensorflow
#!pip install tqdm

### Downloading the Dogs vs Cats dataset 

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


## Dataset preprocessing

### Import project dependencies

In [3]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.3.1'

### Unzipping the Dogs vs Cats dataset

In [4]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [5]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

FileNotFoundError: [Errno 2] No such file or directory: './cats_and_dogs_filtered.zip'

In [ ]:
zip_object.extractall("./")

In [ ]:
zip_object.close()

### Seting up dataset paths

In [ ]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [ ]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model [MobileNetV2](https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2) from [Keras Applications](https://www.tensorflow.org/api_docs/python/tf/keras/applications)  

In [ ]:
IMG_SHAPE = (128, 128, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [ ]:
base_model.summary()

### Freezing the base model

In [ ]:
base_model.trainable = False

### Defining the custom head for our network

In [ ]:
base_model.output

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

In [ ]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary()

### Compiling the model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

In [ ]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

### Training the model

In [ ]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

### Transfer learning model evaluation

In [ ]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

In [ ]:
fine_tune_at = 100

In [ ]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [ ]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

### Evaluating the fine tuned model

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [ ]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))